In [11]:
import torch
import imageio
import skimage.transform
import torchvision
import numpy as np
import os
from torch.utils.data import DataLoader
import datetime
import cv2
import pandas as pd
from torch.utils.data import Dataset
from torch.utils.data import DataLoader
import torchvision.transforms as transforms

from lib import RedNet_data
from lib import RedNet_model
import torch.optim
from lib.utils import utils
from lib.utils.utils import load_ckpt, intersectionAndUnion, AverageMeter, accuracy, macc

In [12]:
class scaleNorm(object):
    def __call__(self, sample):
        image, depth, label = sample['image'], sample['depth'], sample['label']

        # Bi-linear
        image = skimage.transform.resize(image, (image_h, image_w), order=1,
                                         mode='reflect', preserve_range=True)
        # Nearest-neighbor
        depth = skimage.transform.resize(depth, (image_h, image_w), order=0,
                                         mode='reflect', preserve_range=True)
        label = skimage.transform.resize(label, (image_h, image_w), order=0,
                                         mode='reflect', preserve_range=True)

        return {'image': image, 'depth': depth, 'label': label}

class ToTensor(object):
    """Convert ndarrays in sample to Tensors."""

    def __call__(self, sample):
        image, depth, label = sample['image'], sample['depth'], sample['label']

        image = image.transpose((2, 0, 1))
        depth = np.expand_dims(depth, 0).astype(np.float)
        return {'image': torch.from_numpy(image).float(),
                'depth': torch.from_numpy(depth).float(),
                'label': torch.from_numpy(label).float()}

class Normalize(object):
    def __call__(self, sample):
        image, depth = sample['image'], sample['depth']
        origin_image = image.clone()
        origin_depth = depth.clone()
        image = image / 255
        image = torchvision.transforms.Normalize(mean=[0.4850042694973687, 0.41627756261047333, 0.3981809741523051],
                                                 std=[0.26415541082494515, 0.2728415392982039, 0.2831175140191598])(image)
        depth = torchvision.transforms.Normalize(mean=[2.8424503515351494],
                                                 std=[0.9932836506164299])(depth)
        sample['origin_image'] = origin_image
        sample['origin_depth'] = origin_depth
        sample['image'] = image
        sample['depth'] = depth

        return sample

In [13]:
class CARLA(Dataset):
    def __init__(self, transform=None, phase_train=True):
        self.phase_train = phase_train
        self.transform = transform
        tmp = pd.read_csv("data/run1/train.csv")
        self.train_files = tmp["Name"]
        tmp = pd.read_csv("data/run1/test.csv")
        self.test_files = tmp["Name"]

    def __len__(self):
        if self.phase_train:
            return len(self.train_files)
        else:
            return len(self.test_files)

    def __getitem__(self, idx):
        if self.phase_train == True:
            files = self.train_files
        else:
            files = self.test_files
        label = np.load("data/run2/semantic/" + files[idx] + ".npy")
        depth = np.load("data/run2/depth/" + files[idx] + ".npy")
        image = imageio.v2.imread("data/run2/rgb/" + files[idx] + ".png", pilmode='RGB')
        sample = {'image':image, 'depth': depth, 'label':label, "origin_image": image, "origin_depth": depth}
        if self.transform:
            sample = self.transform(sample)
        return sample

In [14]:
checkpoint_path = "lib/models/model1/ckpt_epoch_410.00.pth"
device = torch.device("cuda:0")
image_w = 640
image_h = 480
img_mean=[0.485, 0.456, 0.406]
img_std=[0.229, 0.224, 0.225]

In [15]:
model = RedNet_model.RedNet(pretrained=False)
load_ckpt(model, None, checkpoint_path, device)
model.eval()
model.to(device)
test_data = CARLA(transform=transforms.Compose([scaleNorm(),
                                                 ToTensor(),
                                                 Normalize()]),
                  phase_train=False)
test_loader = DataLoader(test_data, batch_size=1, shuffle=False, num_workers=0, pin_memory=True)
acc_meter = AverageMeter()
intersection_meter = AverageMeter()
union_meter = AverageMeter()
a_meter = AverageMeter()
b_meter = AverageMeter()

=> loading checkpoint 'lib/models/model1/ckpt_epoch_410.00.pth'
=> loaded checkpoint 'lib/models/model1/ckpt_epoch_410.00.pth' (epoch 410.0)


In [19]:
with torch.no_grad():
    for batch_idx, sample in enumerate(test_loader):
        origin_image = sample['origin_image'].numpy()
        origin_depth = sample['origin_depth'].numpy()
        image = sample['image'].to(device)
        depth = sample['depth'].to(device)
        label = sample['label'].numpy()

        with torch.no_grad():
            pred = model(image, depth)

        output = torch.max(pred, 1)[1]
        output = output.squeeze(0).cpu().numpy()

        acc, pix = accuracy(output, label)
        intersection, union = intersectionAndUnion(output, label, 3)
        acc_meter.update(acc, pix)
        a_m, b_m = macc(output, label, 3)
        intersection_meter.update(intersection)
        union_meter.update(union)
        a_meter.update(a_m)
        b_meter.update(b_m)
        print('[{}] iter {}, accuracy: {}'
              .format(datetime.datetime.now().strftime("%Y-%m-%d %H:%M:%S"),
                      batch_idx, acc))

iou = intersection_meter.sum / (union_meter.sum + 1e-10)
for i, _iou in enumerate(iou):
    print('class [{}], IoU: {}'.format(i, _iou))

mAcc = (a_meter.average() / (b_meter.average()+1e-10))
print(mAcc.mean())
print('[Eval Summary]:')
print('Mean IoU: {:.4}, Accuracy: {:.2f}%'
      .format(iou.mean(), acc_meter.average() * 100))

C:\Users\dokee\anaconda3\envs\torch\lib\site-packages\ipykernel_launcher.py:23: DeprecationWarning: `np.float` is a deprecated alias for the builtin `float`. To silence this warning, use `float` by itself. Doing this will not modify any behavior and is safe. If you specifically wanted the numpy scalar type, use `np.float64` here.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations


[2022-09-16 15:08:15] iter 0, accuracy: 0.9155789275988313
[2022-09-16 15:08:15] iter 1, accuracy: 0.9526095345663281
[2022-09-16 15:08:15] iter 2, accuracy: 0.9587321742527791
[2022-09-16 15:08:15] iter 3, accuracy: 0.9951970201921144
[2022-09-16 15:08:16] iter 4, accuracy: 0.8127327327327295
[2022-09-16 15:08:16] iter 5, accuracy: 0.9535864978902909
[2022-09-16 15:08:16] iter 6, accuracy: 0.9994559303590811
[2022-09-16 15:08:16] iter 7, accuracy: 0.6335505937015984
[2022-09-16 15:08:16] iter 8, accuracy: 0.9625334957369016
[2022-09-16 15:08:16] iter 9, accuracy: 0.9182269237970375
[2022-09-16 15:08:17] iter 10, accuracy: 0.9619206085096224
[2022-09-16 15:08:17] iter 11, accuracy: 0.9718289786223232
[2022-09-16 15:08:17] iter 12, accuracy: 0.9352723915050742
[2022-09-16 15:08:17] iter 13, accuracy: 0.9995053667705348
[2022-09-16 15:08:17] iter 14, accuracy: 0.9219205539358559
[2022-09-16 15:08:17] iter 15, accuracy: 0.8088951201464315
[2022-09-16 15:08:18] iter 16, accuracy: 0.9994064

[2022-09-16 15:08:41] iter 137, accuracy: 0.952180793195525
[2022-09-16 15:08:41] iter 138, accuracy: 0.9816117607102957
[2022-09-16 15:08:41] iter 139, accuracy: 0.9503009215723108
[2022-09-16 15:08:41] iter 140, accuracy: 0.9967011324470656
[2022-09-16 15:08:42] iter 141, accuracy: 0.9646679441043092
[2022-09-16 15:08:42] iter 142, accuracy: 0.9336558068952392
[2022-09-16 15:08:42] iter 143, accuracy: 0.9457870027180582
[2022-09-16 15:08:42] iter 144, accuracy: 0.7118192026951127
[2022-09-16 15:08:42] iter 145, accuracy: 0.9738182847438392
[2022-09-16 15:08:43] iter 146, accuracy: 0.9856090541001953
[2022-09-16 15:08:43] iter 147, accuracy: 0.9999999999999951
[2022-09-16 15:08:43] iter 148, accuracy: 0.9598435217042893
[2022-09-16 15:08:43] iter 149, accuracy: 0.9605316880642338
[2022-09-16 15:08:43] iter 150, accuracy: 0.9582137311185311
[2022-09-16 15:08:44] iter 151, accuracy: 0.9989613215946138
[2022-09-16 15:08:44] iter 152, accuracy: 0.9998515365962242
[2022-09-16 15:08:44] ite